In [63]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

info = pd.read_excel("spx_returns_weekly.xlsx", sheet_name="s&p500 names")
info.set_index('ticker',inplace=True)
TICKS = ["AAPL", "NVDA", "MSFT", "GOOGL", "AMZN", "META", "TSLA", "AVGO", "BRK/B", "LLY"]
TICK_ETF = "SPY"
info.loc[TICKS]

rets = pd.read_excel("spx_returns_weekly.xlsx",sheet_name= "s&p500 rets")
rets.set_index('date',inplace=True)
rets = rets[TICKS]

bench = pd.read_excel("spx_returns_weekly.xlsx", sheet_name="benchmark rets")
bench.set_index('date',inplace=True)

rets[TICK_ETF] = bench[TICK_ETF]

In [64]:
FREQ = 52
TARGET_MEAN = 0.2 #anualzied target mean
mean = rets.mean() * FREQ #annualized mean
cov = rets.cov() * FREQ #annualized covariance matrix
n_assets = len(TICKS) + 1
initial_weights = np.ones(n_assets) / n_assets #equal weights


In [65]:
#1.1 Constrained Portfolio
def objective(w):        
    return (w.T @ cov @ w)

def fun_constraint_capital(w):
    return np.sum(w) - 1

def fun_constraint_mean(w):
    return (mean @ w) - TARGET_MEAN

constraint_capital = {'type': 'eq', 'fun': fun_constraint_capital}
constraint_mean = {'type': 'eq', 'fun': fun_constraint_mean}

constraints = ([constraint_capital, constraint_mean])

#Incorporate bounds:
bounds = [(-0.2,0.35) for _ in range(n_assets)]

portfolio = minimize(objective, initial_weights, method = "SLSQP",
                     bounds = bounds, constraints = constraints)

optimal_weights = portfolio.x
portfolio_var = portfolio.fun
portfolio_vol = np.sqrt(portfolio_var)
portfolio_mean = mean @ optimal_weights
portfolio_sharpe = portfolio_mean / portfolio_vol

weights = pd.DataFrame({
    "Ticker" : TICKS + [TICK_ETF],
    "Weight" : optimal_weights.round(5)
})

weights

,Ticker,Weight
0,AAPL,0.02924
1,NVDA,-0.01434
2,MSFT,0.14451
3,GOOGL,0.00906
4,AMZN,0.09386
5,META,0.00287
6,TSLA,-0.01507
7,AVGO,0.03625
8,BRK/B,0.35000
9,LLY,0.21308


In [66]:
portfolio_results = pd.DataFrame({
    "Mean" : [portfolio_mean],
    "Volatility": [portfolio_vol],
    "Sharpe Ratio" : [portfolio_sharpe]
})
print("Constrained Portfolio Results:")
portfolio_results

Constrained Portfolio Results:


,Mean,Volatility,Sharpe Ratio
0,0.2,0.164889,1.212936


In [67]:
#1.2 Weights vs. Assets' Sharpe & Means

indiv_vol = np.sqrt(np.diag(cov))
indiv_sharpe = mean / indiv_vol
    

compare_df = pd.DataFrame({
    "Ticker" : TICKS + [TICK_ETF],
    "Weight" : optimal_weights.round(5),
    "Sharpe Ratio": indiv_sharpe.round(5),
    "Mean" : mean.round(5)
})
sorted_comparison = compare_df.sort_values("Weight", ascending = False).set_index("Ticker")
sorted_comparison

,Weight,Sharpe Ratio,Mean
Ticker,,,
BRK/B,0.35000,0.70822,0.13503
LLY,0.21308,0.99492,0.28154
SPY,0.15053,0.76818,0.13126
MSFT,0.14451,1.08927,0.26140
AMZN,0.09386,0.95898,0.29345
AVGO,0.03625,1.05257,0.39485
AAPL,0.02924,0.86294,0.23871
GOOGL,0.00906,0.77470,0.21680
META,0.00287,0.74551,0.26192


###
From the table above, it does not look like there's a relationship between the assets' weight and their Sharpe & mean. For example, the asset with the highest weight, BRK/B, does not have the highest Sharpe nor the highest mean. Similarly, the asset with the lowest weight, TSLA, does not have the lowst Sharpe nor the lowest mean. 

This finding is a result of the constrained optimization

In [68]:
#1.3 Bounded vs Unbounded

#unbounded performance:
unbounds = None
unbounded_result = minimize(objective, initial_weights, method = "SLSQP",
                            bounds = unbounds, constraints = constraints)

unbounded_weights = unbounded_result.x
unbounded_var = unbounded_result.fun
unbounded_vol = np.sqrt(unbounded_var)
unbounded_mean = mean @ unbounded_weights
unbounded_sharpe = unbounded_mean / unbounded_vol

#Compare Performances:
bound_vs_unbound = pd.DataFrame({
    "Ticker" : TICKS + [TICK_ETF],
    "Bounded Weight" : optimal_weights.round(5),
    "Ubounded Weight": unbounded_weights.round(5),
    "Difference": (unbounded_weights - optimal_weights).round(5)
}).sort_values("Bounded Weight", ascending = False)

bound_vs_unbound

,Ticker,Bounded Weight,Ubounded Weight,Difference
8,BRK/B,0.35000,0.36759,0.01759
9,LLY,0.21308,0.21053,-0.00256
10,SPY,0.15053,0.13009,-0.02044
2,MSFT,0.14451,0.15200,0.00750
4,AMZN,0.09386,0.09493,0.00108
7,AVGO,0.03625,0.03545,-0.00080
0,AAPL,0.02924,0.02947,0.00022
3,GOOGL,0.00906,0.00821,-0.00085
5,META,0.00287,0.00197,-0.00091
1,NVDA,-0.01434,-0.01533,-0.00098


In [69]:
#mean, vol, sharpe comparison:

bound_vs_unbound_stats = pd.DataFrame({
    "Mean" : [portfolio_mean, unbounded_mean],
    "Volatility": [portfolio_vol, unbounded_vol],
    "Sharpe Ratio" : [portfolio_sharpe, unbounded_sharpe]
}, index = ["Bounded", "Unbounded"])

bound_vs_unbound_stats

,Mean,Volatility,Sharpe Ratio
Bounded,0.2,0.164889,1.212936
Unbounded,0.2,0.164866,1.213103


### Comparison of Bounded & Unbounded Portfolios:

From the statistics above, we see that the Bounded & Unbounded Portfolios had very similar performances.